In [1]:
from time import sleep

import requests
import json
import pandas as pd
import time
from collections import defaultdict
from datetime import datetime

from dotenv import load_dotenv
import os


In [2]:
load_dotenv(dotenv_path='app.env')

token = os.getenv('GITHUB_TOKEN')

In [65]:
#Output
folder_path = "Data/50_fin"



json_file = f"{folder_path}/append_dependencies_50_final.json"
csv_file = f"{folder_path}/github_50_projects_with_topics_1.csv"

In [45]:
# Read the CSV file
file_path = 'Data/Application_types.csv'
df = pd.read_csv(file_path)

# Convert the 'Keyword' column to a list
topic_list = df['Keyword'].tolist()

# Display the list
print(topic_list)


def write_json_data(fileName, data):
    with open(fileName, 'w') as file:
        json.dump(data, file, indent=4)


def append_json_data(fileName, data):
    # Try to read existing data if the file exists
    try:
        with open(fileName, 'r') as file:
            existing_data = json.load(file)
    except FileNotFoundError:
        existing_data = []  # Initialize as an empty list if the file doesn't exist

    # Append new data to existing data
    existing_data.append(data)

    # Write the updated data back to the file
    with open(fileName, 'w') as file:
        json.dump(existing_data, file, indent=4)


def sleep_if_rate_exhausted(response):
    core_limits = response.headers._store
    rate_left = core_limits['x-ratelimit-remaining'][1]

    if int(rate_left) > 0:
        return

    reset_time = core_limits['x-ratelimit-reset'][1]
    reset_timestamp = datetime.fromtimestamp(int(reset_time))
    current_time = datetime.now()
    sleep_duration = (reset_timestamp - current_time).total_seconds()
    print(f"Rate limit reached. Sleeping until reset at {reset_timestamp}")
    time.sleep(sleep_duration+5)  # Sleep until reset time + some delay time
    print("Rate limit reset. Resuming requests.")


['Dashboard', 'CMS', 'E-commerce', 'Portfolio', 'Blog', 'Chatbot', 'Social Network', 'Messaging', 'Forum', 'Community', 'News', 'Wiki', 'File Manager', 'Analytics', 'Project Management', 'Task Manager', 'Calendar', 'Notes', 'Kanban', 'ERP', 'CRM', 'POS', 'Inventory', 'To-do', 'Video Streaming', 'Music Player', 'Photo Gallery', 'Weather', 'Finance', 'Budget Tracker', 'Expense Tracker', 'Banking', 'Travel', 'Booking', 'Reservation', 'Hotel Management', 'Real Estate', 'Job Portal', 'Classroom', 'Learning Management System', 'Online Exam', 'Survey', 'Poll', 'E-learning', 'Quiz', 'Recipe', 'Cookbook', 'Portfolio', 'Resume Builder', 'Event Management', 'Gaming', 'Game Tracker', 'Game Leaderboard', 'VR', 'AR', '3D Modeling', 'Fitness', 'Workout Tracker', 'Health Tracker', 'Diet Planner', 'Meal Planner', 'Meditation', 'Habit Tracker', 'Productivity', 'Chat', 'Collaboration', 'Help Desk', 'Customer Support', 'Feedback', 'Documentation', 'Startup', 'Portfolio', 'Map', 'Geo-location', 'Travel Gui

In [48]:
def get_github_projects_of_topic(num_results,num_pages, topic):
    headers = {'Authorization': f'token {token}'}
    url = 'https://api.github.com/search/repositories'
    params = {
        # 'q': f'(language:JavaScript OR language:TypeScript) {topic} stars:>50',
        'q': f'language:JavaScript {topic} stars:>50',
        'sort': 'stars',
        'order': 'desc',
        'per_page': num_results,
        'page': num_pages
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        projects = [
            {
                'id': item['id'],
                'name': f"{item['name']}+{item['owner']['login']}",  # Combine name and owner
                'owner': item['owner']['login'],  # Keep the owner column
                'watchers': item['watchers_count'],
                'created_at': item['created_at'],
                'updated_at': item['updated_at'],
                'size': item['size'],
                'stars': item['stargazers_count'],
                'open_issues_count': item['open_issues_count'],
                'language': item['language'],
                'forks': item['forks'],
                'description': item['description'],
                'topics': item['topics']
            } 
            for item in data['items']
        ]
        
        
        
        for project in projects:
            if project['id'] in projects_set:
                continue
            
            unique_projects.append(project)
            projects_set.add(project['id'])
            if(len(unique_projects) >= total_num_results):
                break

        

        return unique_projects, response,len(unique_projects),len(projects)

    else:
        print(f"Error: {response.status_code}")
        return None, response

topic_dict = {}
num_topics = 50
projects_set = set()
for i in range(num_topics):
    currentTopic = topic_list[i]
    num_unique_results = 0
    num_pages=1
    unique_projects = []
    prev_results=0
    num_results = 10
    total_num_results = num_results
    while True:
        projects, response, num_unique_results, num_of_results_from_API = get_github_projects_of_topic(num_results=num_results, num_pages=num_pages,topic=currentTopic)
        
        if projects is not None:
            topic_dict[currentTopic] = projects
        
        if num_unique_results >= total_num_results:
            break
        
        if num_of_results_from_API == prev_results:
            break
        
        prev_results = num_of_results_from_API
        
        if num_results<100:
            num_results+=10
        else:
            num_pages+=1
        
        
        sleep_if_rate_exhausted(response)
        
    sleep_if_rate_exhausted(response)
    
        
    
        

print(topic_dict)








Rate limit reached. Sleeping until reset at 2024-11-20 16:53:18
Rate limit reset. Resuming requests.
Rate limit reached. Sleeping until reset at 2024-11-20 16:54:23
Rate limit reset. Resuming requests.
{'Dashboard': [{'id': 528313152, 'name': 'homepage+gethomepage', 'owner': 'gethomepage', 'watchers': 19716, 'created_at': '2022-08-24T07:29:42Z', 'updated_at': '2024-11-20T08:50:13Z', 'size': 48141, 'stars': 19716, 'open_issues_count': 1, 'language': 'JavaScript', 'forks': 1178, 'description': 'A highly customizable homepage (or startpage / application dashboard) with Docker and service API integrations.', 'topics': ['docker', 'homepage', 'nextjs', 'node', 'react', 'self-hosted', 'startpage']}, {'id': 29247444, 'name': 'blessed-contrib+yaronn', 'owner': 'yaronn', 'watchers': 15498, 'created_at': '2015-01-14T14:18:07Z', 'updated_at': '2024-11-19T10:24:11Z', 'size': 5490, 'stars': 15498, 'open_issues_count': 96, 'language': 'JavaScript', 'forks': 840, 'description': 'Build terminal dashboa

In [52]:
def get_Sbom_of_projects(repo_df):
    headers = {'Authorization': f'token {token}'}
    url = 'https://api.github.com/repos/'
    owner = repo_df['owner']
    repo_name_owner = repo_df['name']
    repo = repo_df['name'].split('+')[0]
    url = url + f'{owner}/{repo}/dependency-graph/sbom'

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        packages = data['sbom']['packages']
        # Extract npm packages with name and version
        pkg_dict = defaultdict(dict)
        data_written = False
        for package in packages:
            if package['externalRefs'][0]['referenceLocator'].startswith('pkg:npm'):
                try:
                    if len(package['versionInfo']) > 0:
                        if 'versionInfo' in package:
                            if package['name'].startswith('npm:'):
                                data_written = True
                                sbom_dict[repo_name_owner][package['name'][4:]] = package['versionInfo']
                            else:
                                data_written = True
                                sbom_dict[repo_name_owner][package['name']] = package['versionInfo']
                except:
                    continue
        
        if not data_written:
            print(repo_df)
            print(packages[0]['externalRefs'][0]['referenceLocator'])
            remove_projects.append(repo_df)
    else:
        print(f"Error: {response.status_code}")
        remove_projects.append(repo_df)

    return response


remove_projects = []
sbom_dict = defaultdict(dict)
ctr = tctr = 1
for topic, projects in topic_dict.items():
    for project in projects:
        response = get_Sbom_of_projects(project)
        sleep_if_rate_exhausted(response)
    print(f'{ctr}. {topic} done')
    ctr+=1


os.makedirs(folder_path, exist_ok=True)
write_json_data(json_file, sbom_dict)

{'id': 5101823, 'name': 'dashing+Shopify', 'owner': 'Shopify', 'watchers': 10973, 'created_at': '2012-07-18T21:01:40Z', 'updated_at': '2024-11-19T08:23:17Z', 'size': 2543, 'stars': 10973, 'open_issues_count': 251, 'language': 'JavaScript', 'forks': 1178, 'description': 'The exceptionally handsome dashboard framework in Ruby and Coffeescript.', 'topics': []}
pkg:gem/dashing
1. Dashboard done
2. CMS done
3. E-commerce done
4. Portfolio done
{'id': 82329147, 'name': 'blog+jawil', 'owner': 'jawil', 'watchers': 7890, 'created_at': '2017-02-17T19:06:54Z', 'updated_at': '2024-11-07T06:30:25Z', 'size': 79, 'stars': 7890, 'open_issues_count': 28, 'language': 'JavaScript', 'forks': 840, 'description': 'Too young, too simple. Sometimes, naive & stupid 🐌', 'topics': ['blog', 'css', 'difficulty', 'es6', 'github', 'javascript', 'js', 'nodejs', 'notes', 'study']}
pkg:github/jawil/blog@master
{'id': 152988607, 'name': 'hexo-theme-fluid+fluid-dev', 'owner': 'fluid-dev', 'watchers': 7363, 'created_at': 

FileNotFoundError: [Errno 2] No such file or directory: 'Data/50_final/github_50_projects_with_topics_1.csv'

In [66]:
os.makedirs(folder_path, exist_ok=True)
write_json_data(json_file, sbom_dict)

In [67]:
print(len(remove_projects))


def check_if_in_remove(project_id):
    for i in remove_projects:
        if i['id'] == project_id:
            return True
    return False

58


In [68]:
all_projects = []

filename = csv_file
for topic, projects in topic_dict.items():
    for project in projects:
        # Add the topic as a new field in each project dictionary
        if(not (check_if_in_remove(project["id"]))):
            project['topic'] = topic
            all_projects.append(project)
        else:
            print(project)

# Convert the list of all projects with topics into a DataFrame
df = pd.DataFrame(all_projects)

# Save the DataFrame to a CSV file
df.to_csv(f'{filename}', index=False)

print(f"Data saved to {filename}.csv")

error_from = ""

{'id': 5101823, 'name': 'dashing+Shopify', 'owner': 'Shopify', 'watchers': 10973, 'created_at': '2012-07-18T21:01:40Z', 'updated_at': '2024-11-19T08:23:17Z', 'size': 2543, 'stars': 10973, 'open_issues_count': 251, 'language': 'JavaScript', 'forks': 1178, 'description': 'The exceptionally handsome dashboard framework in Ruby and Coffeescript.', 'topics': []}
{'id': 82329147, 'name': 'blog+jawil', 'owner': 'jawil', 'watchers': 7890, 'created_at': '2017-02-17T19:06:54Z', 'updated_at': '2024-11-07T06:30:25Z', 'size': 79, 'stars': 7890, 'open_issues_count': 28, 'language': 'JavaScript', 'forks': 840, 'description': 'Too young, too simple. Sometimes, naive & stupid 🐌', 'topics': ['blog', 'css', 'difficulty', 'es6', 'github', 'javascript', 'js', 'nodejs', 'notes', 'study']}
{'id': 152988607, 'name': 'hexo-theme-fluid+fluid-dev', 'owner': 'fluid-dev', 'watchers': 7363, 'created_at': '2018-10-14T15:26:24Z', 'updated_at': '2024-11-20T08:28:52Z', 'size': 8486, 'stars': 7363, 'open_issues_count': 

In [69]:
json_data = json.load(open(json_file))

print(len(json_data.keys()))


405


In [71]:
file_path = csv_file
csv_df = pd.read_csv(file_path)

print(len(csv_df))

csv_df.iterrows()

# for i, data in  csv_df.iterrows():
#     if(data["name"] not in json_data.keys()):
#         print(data["name"])






405


<generator object DataFrame.iterrows at 0x00000270F0455030>